In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import cv2
import numpy as np

In [ ]:
%run ResNet34Model.ipynb
%run DatasetStatistics.ipynb
%run HistogramEqualization.ipynb

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
checkpoint = torch.load("./config/resnet34.pth", map_location=device)

In [ ]:
IMAGE_SIZE = checkpoint["image_size"]
MEAN = checkpoint["mean"] / 255.0
STD = checkpoint["std"] / 255.0
DIR = "./new_data/Testing"

In [ ]:
transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        HistogramEqualization(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[MEAN] * 3, std=[STD] * 3),
    ]
)

In [ ]:
dataset = datasets.ImageFolder(DIR, transform=transforms)

In [ ]:
dl = DataLoader(dataset, shuffle=False, batch_size=1)

In [ ]:
model = ResNet34Model()
model.load_state_dict(checkpoint["weights"])
model.to(device)
model.eval()

In [ ]:
img, _ = next(iter(dl))
img = img.to(device)

In [ ]:
out = model(img)

In [ ]:
class_index = out.argmax(dim=1).item()

In [ ]:
model.zero_grad()
loss = out[:, class_index].backward()

In [ ]:
grads = model.get_activations_gradient()
pool_gradients = torch.mean(grads, dim=[0, 2, 3])

In [ ]:
activations = model.get_activations()

for i in range(512):
    activations[:, i, :, :] *= pool_gradients[i]

In [ ]:
heatmap = torch.mean(activations, dim=1).squeeze()
heatmap = F.relu(heatmap)  # reLU

In [ ]:
heatmap /= heatmap.max()  # normalize

In [ ]:
heatmap = heatmap.cpu().numpy()
plt.matshow(heatmap.squeeze())

In [ ]:
img_path, _ = dataset.samples[0]
img_original = cv2.imread(img_path)
img_original = cv2.resize(img_original, IMAGE_SIZE)

In [ ]:
heatmap = cv2.resize(heatmap, (img_original.shape[1], img_original.shape[0]))
heatmap = (heatmap * 255).astype(np.uint8)

In [ ]:
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

In [ ]:
superimposed_img = cv2.addWeighted(img_original, 0.6, heatmap, 0.4, 0)

In [ ]:
# TODO: add visualization